In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Dataset
Dataset for this problem:

<img src="images/dataset.png" alt="drawing" width="500"/>

In [2]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [3]:
print("First few elements of X_train:\n", X_train[:5])
print("Shaep of X_train:",X_train.shape)

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Shaep of X_train: (10, 3)


In [4]:
print("First few elements of y_train:", y_train[:5])
print("Shape of y_train:",y_train.shape)

First few elements of y_train: [1 1 0 0 1]
Shape of y_train: (10,)


## Implementing Decision Tree

### 1. Calculate Entropy

In [5]:
def compute_entropy(y):
    """
    Computes the entropy for node
    
    Args:
      y (ndarray)  : Numpy array indicating whether each example at a node is
                     edible ('1') or poisonous ('0')
                     
    Returns:
      entropy (float)  : Entropy at that node
    """

    entropy = 0.

    if len(y) != 0:
        p_1 = sum(y) / len(y)

        if p_1 == 1 or p_1 == 0:
            entropy = 0
        else:
            entropy = (-p_1 * np.log2(p_1)) - ((1 - p_1) * np.log2(1 - p_1))

    return entropy

In [6]:
print("Entropy at root node: ", compute_entropy(y_train)) 

Entropy at root node:  1.0


### 2. Split Dataset

In [7]:
def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):     List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (list):     Indices with feature value == 1
        right_indices (list):    Indices with feature value == 0
    """

    left_indices = []
    right_indices = []
    
    for i,x in enumerate(X[node_indices]):
        if x[feature] == 1:
            left_indices.append(node_indices[i])
        else:
            right_indices.append(node_indices[i])
        
    return left_indices, right_indices

In [8]:
# Case 1

root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("CASE 1:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

CASE 1:
Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]


In [9]:
# Case 2

root_indices_subset = [0, 2, 4, 6, 8]
left_indices, right_indices = split_dataset(X_train, root_indices_subset, feature)

print("CASE 2:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

CASE 2:
Left indices:  [0, 2, 4]
Right indices:  [6, 8]


### 3. Calculate information gain

In [10]:
def compute_information_gain(X, y, node_indices, feature):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        feature (int):           Index of feature to split on
   
    Returns:
        cost (float):        Cost computed
    
    """    
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    information_gain = 0
    
    H_node = compute_entropy(y_node)
    H_left = len(y_left) / len(y_node) * compute_entropy(y_left)
    H_right = len(y_right) / len(y_node) * compute_entropy(y_right)
    
    information_gain = H_node - H_left - H_right
    
    return information_gain

In [11]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

Information Gain from splitting the root on brown cap:  0.03485155455967709
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365324
Information Gain from splitting the root on solitary:  0.2780719051126377


### 4. Get best split

In [12]:
def get_best_split(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    

    num_features = X.shape[1]
    
    best_feature = -1
    list_feature_information_gain = []
    
    for feature in range(num_features):
        list_feature_information_gain.append(compute_information_gain(X, y, node_indices, feature))
    
    if np.max(list_feature_information_gain) > 0:
        best_feature = list_feature_information_gain.index(np.max(list_feature_information_gain))
   
    return best_feature

In [13]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


In [14]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
        
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [15]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
